In [1]:
import pandas as pd, numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Dữ liệu nhỏ (4 user × 4 item)
R = pd.DataFrame(
    [[4,3,np.nan,4],
     [3,3,5,5],
     [5,4,3,4],
     [np.nan,2,3,5]],
    index=['    U1','U2','U3','U4'], columns=['I1','I2','I3','I4']
)
R


,I1,I2,I3,I4
U1,4.0,3,NaN,4
U2,3.0,3,5.0,5
U3,5.0,4,3.0,4
U4,NaN,2,3.0,5


## 1.Tính ma trận tương đồng
- (a) Giữa user–user

In [2]:
# Điền NaN = 0 (hoặc mean user)
R_user = R.fillna(0)
sim_user = pd.DataFrame(cosine_similarity(R_user), 
                        index=R.index, columns=R.index)

print("🎯 Ma trận tương đồng giữa các USER:")
print(sim_user.round(3))


🎯 Ma trận tương đồng giữa các USER:
            U1     U2     U3     U4
    U1   1.000  0.776  0.923  0.659
U2       0.776  1.000  0.925  0.905
U3       0.923  0.925  1.000  0.739
U4       0.659  0.905  0.739  1.000


- 🔹 (b) Giữa item–item

In [3]:
R_item = R.fillna(0).T
sim_item = pd.DataFrame(cosine_similarity(R_item),
                        index=R.columns, columns=R.columns)

print("\n🎯 Ma trận tương đồng giữa các ITEM:")
print(sim_item.round(3))



🎯 Ma trận tương đồng giữa các ITEM:
       I1     I2     I3     I4
I1  1.000  0.941  0.647  0.796
I2  0.941  1.000  0.816  0.949
I3  0.647  0.816  1.000  0.876
I4  0.796  0.949  0.876  1.000


## 2.User-based Collaborative Filtering

Ý tưởng: dự đoán rating còn thiếu của user dựa trên user tương tự.

$$\hat r_{u,i} = \frac{\sum_v s(u,v),r_{v,i}}{\sum_v |s(u,v)|}$$
	​


In [4]:
def predict_user_based(u, i, k=2):
    if not np.isnan(R.loc[u, i]): 
        return R.loc[u, i]
    sims = sim_user.loc[u].drop(u)
    topk = sims.nlargest(k)
    num = den = 0
    for v, s in topk.items():
        if not np.isnan(R.loc[v, i]):
            num += s * R.loc[v, i]
            den += abs(s)
    return np.clip(num/den, 1, 5) if den else np.nan

# Dự đoán toàn bộ
pred_user = R.copy()
for u in R.index:
    for i in R.columns:
        if pd.isna(pred_user.loc[u, i]):
            pred_user.loc[u, i] = predict_user_based(u, i)

print("\n📊 Ma trận dự đoán (User-based CF):")
print(pred_user.round(3))



📊 Ma trận dự đoán (User-based CF):
           I1  I2     I3  I4
    U1  4.000   3  3.914   4
U2      3.000   3  5.000   5
U3      5.000   4  3.000   4
U4      3.899   2  3.000   5


## 3.Gợi ý item cho từng user

In [5]:
print("\n🎬 Gợi ý cho từng USER:")
for u in R.index:
    unrated = R.loc[u][R.loc[u].isna()].index
    if len(unrated)==0: continue
    rec = {i: predict_user_based(u,i) for i in unrated}
    best = max(rec, key=rec.get)
    print(f"👉 {u} nên xem {best} (dự đoán = {rec[best]:.2f})")



🎬 Gợi ý cho từng USER:
👉     U1 nên xem I3 (dự đoán = 3.91)
👉 U4 nên xem I1 (dự đoán = 3.90)


## 4.Item-based Collaborative Filtering

$$\hat r_{u,i} = \frac{\sum_j s(i,j)r_{u,j}}{\sum_j |s(i,j)|}$$

In [6]:
def predict_item_based(u, i, k=2):
    if not np.isnan(R.loc[u,i]):
        return R.loc[u,i]
    sims = sim_item.loc[i].drop(i)
    topk = sims.nlargest(k)
    num = den = 0
    for j, s in topk.items():
        if not np.isnan(R.loc[u,j]):
            num += s * R.loc[u,j]
            den += abs(s)
    return np.clip(num/den, 1, 5) if den else np.nan

pred_item = R.copy()
for u in R.index:
    for i in R.columns:
        if pd.isna(pred_item.loc[u,i]):
            pred_item.loc[u,i] = predict_item_based(u,i)

print("\n📈 Ma trận dự đoán (Item-based CF):")
print(pred_item.round(3))



📈 Ma trận dự đoán (Item-based CF):
           I1  I2     I3  I4
    U1  4.000   3  3.518   4
U2      3.000   3  5.000   5
U3      5.000   4  3.000   4
U4      3.376   2  3.000   5


## 5.Gợi ý user cho từng item

In [7]:
print("\n🎯 Gợi ý USER cho từng ITEM:")
for i in R.columns:
    missing_users = R[i][R[i].isna()].index
    if len(missing_users)==0: continue
    rec = {u: predict_item_based(u,i) for u in missing_users}
    best_user = max(rec, key=rec.get)
    print(f"🎬 Item {i} nên gợi ý cho {best_user} (dự đoán = {rec[best_user]:.2f})")



🎯 Gợi ý USER cho từng ITEM:
🎬 Item I1 nên gợi ý cho U4 (dự đoán = 3.38)
🎬 Item I3 nên gợi ý cho     U1 (dự đoán = 3.52)


## 6.So sánh kết quả

In [8]:
print("\n📊 So sánh nhanh:")
print("User-based:")
print(pred_user.round(2))
print("\nItem-based:")
print(pred_item.round(2))



📊 So sánh nhanh:
User-based:
         I1  I2    I3  I4
    U1  4.0   3  3.91   4
U2      3.0   3  5.00   5
U3      5.0   4  3.00   4
U4      3.9   2  3.00   5

Item-based:
          I1  I2    I3  I4
    U1  4.00   3  3.52   4
U2      3.00   3  5.00   5
U3      5.00   4  3.00   4
U4      3.38   2  3.00   5


## 7.Mẫu code rút gọn (nếu đề yêu cầu “chạy nhanh trong 5 dòng”)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
R_filled = R.fillna(0)
sim_u = pd.DataFrame(cosine_similarity(R_filled), index=R.index, columns=R.index)
sim_i = pd.DataFrame(cosine_similarity(R_filled.T), index=R.columns, columns=R.columns)
print("User sim:\n", sim_u.round(2))
print("\nItem sim:\n", sim_i.round(2))


User sim:
             U1    U2    U3    U4
    U1    1.00  0.78  0.92  0.66
U2        0.78  1.00  0.93  0.90
U3        0.92  0.93  1.00  0.74
U4        0.66  0.90  0.74  1.00

Item sim:
       I1    I2    I3    I4
I1  1.00  0.94  0.65  0.80
I2  0.94  1.00  0.82  0.95
I3  0.65  0.82  1.00  0.88
I4  0.80  0.95  0.88  1.00


: 

# Lý Thuyết

| Phân tích    | Nhận xét                                                                                                |
| ------------ | ------------------------------------------------------------------------------------------------------- |
| Độ chính xác | User-based và item-based thường khá gần nhau nếu dữ liệu đầy đủ.                                        |
| Tính ổn định | Item-based ổn định hơn vì item không thay đổi thường xuyên, trong khi user mới xuất hiện liên tục.      |
| Tốc độ       | Item-based nhanh hơn khi hệ thống nhiều user.                                                           |
| Khuyến nghị  | Kết quả khuyến nghị cho user và item có thể trùng nhau về logic (phim được nhiều người tương tự thích). |
| Cold-start   | User-based bị ảnh hưởng nặng khi user mới; item-based chịu khi item mới.                                |


| **Nhóm biến thể**                      | **Mô tả đề thay đổi**                       | **Cách làm / Code áp dụng**                     |
| -------------------------------------- | ------------------------------------------- | ----------------------------------------------- |
| 🔹 **Thay Cosine bằng Pearson**        | “Tính tương đồng Pearson giữa user & item.” | Dùng `np.corrcoef` thay `cosine_similarity`.    |
| 🔹 **Thêm k-NN = 3,5**                 | “Chỉ xét 3 user gần nhất.”                  | Thay tham số `k`.                               |
| 🔹 **Chia dữ liệu train/test**         | “Tách 20% rating để kiểm tra.”              | Dùng train-test split + RMSE.                   |
| 🔹 **Chuẩn hoá mean trước similarity** | “Trừ trung bình mỗi user.”                  | Sử dụng centered-cosine (như bài 3).            |
| 🔹 **Đánh giá Precision@K**            | “Tính độ chính xác top 5 gợi ý.”            | So sánh item gợi ý vs item user đã thích.       |
| 🔹 **Trực quan hoá**                   | “Vẽ heatmap similarity.”                    | Dùng seaborn: `sns.heatmap(sim_user)`           |
| 🔹 **Dùng MovieLens thật**             | “Dataset lớn hơn.”                          | Dùng pivot: `df.pivot(userId, movieId, rating)` |
| 🔹 **Dự đoán toàn ma trận**            | “Hoàn thiện tất cả ô thiếu.”                | Vòng for toàn R.                                |
| 🔹 **Item-group recommendation**       | “Nhóm item tương tự thành cluster.”         | Dùng KMeans trên `sim_item`.                    |


| Dạng                                            | Ý tưởng / Đề bài có thể gặp         | Hướng xử lý                      |
| ----------------------------------------------- | ----------------------------------- | -------------------------------- |
| 🧠 **Weighted Hybrid**                          | “Kết hợp user-based và item-based.” | (\hat r = αr_{UB} + (1−α)r_{IB}) |
| 🧩 **Matrix Factorization (SVD)**               | “Dự đoán bằng decomposition.”       | Dùng `TruncatedSVD`.             |
| 🧮 **K-Nearest Neighbors CF (scikit-surprise)** | “Dùng thư viện surprise.”           | `KNNBasic`, `KNNWithMeans`.      |
| ⚗️ **Implicit feedback (binary)**               | “Chỉ biết user xem hay không.”      | Dùng ALS / BPR.                  |
| 🔍 **Top-N Recommendation**                     | “In ra top 5 item cho mỗi user.”    | Sort dự đoán từng hàng.          |
